In [ ]:
#%%capture
import os
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive',
            force_remount=True
            )


notebookname = 'SEO.ipynb'

class github:
    def __init__(self, github_pat, github_email, github_username, github_repo, gdrive_notebook_folder, notebook_name):
        self.github_pat = userdata.get(github_pat)
        self.github_email = userdata.get(github_email)
        self.github_username = userdata.get(github_username)
        self.github_repo = userdata.get(github_repo)
        self.gdrive_notebook_folder = userdata.get(gdrive_notebook_folder)
        self.notebook_name = notebook_name

    def clone_repo(self):
        # Source file path in Google Drive
        source_file_path = f"/content/drive/MyDrive/{self.gdrive_notebook_folder}/{self.notebook_name}"

        # Repository details
        repo_url = f'https://{self.github_pat}@github.com/{self.github_username}/{self.github_repo}.git'

        # Clone the private repository
        !git clone {repo_url} cloned-repo
        os.chdir('cloned-repo')  # Switch to the cloned repository

        # Ensure the file exists in Google Drive
        if os.path.exists(source_file_path):
            # Copy the notebook into the cloned repository
            !cp "{source_file_path}" ./
        else:
            print(f"The file {source_file_path} was not found.")
            return  # Exit if the file doesn't exist

        # Git configuration
        !git config user.email "{self.github_email}"
        !git config user.name "{self.github_username}"

        # Add the file to Git
        !git add "{self.notebook_name}"

        # Commit the changes
        !git commit -m "Added {self.notebook_name} from Google Drive"

        # Push to the repository
        !git push origin main

        # Wechsle zurück ins übergeordnete Verzeichnis und lösche cloned-repo
        os.chdir('..')
        !rm -rf cloned-repo
        print("cloned-repo wurde wieder gelöscht.")



# Clone, add, and push the notebook
clone_2 = github('github_pat', 'github_email', 'github_username', 'github_repo_seo', 'gdrive_seo_folder', notebookname)
clone_2.clone_repo()


In [9]:
import os
import requests
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin, urlparse
import time
from google.colab import userdata
import openai
os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_api_key')



class Chatbot:

    def __init__(self, systemprompt, prompt):
        # Neue Client-Initialisierung
        self.client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])
        self.systemprompt = systemprompt
        self.prompt = prompt
        self.context = [{"role": "system", "content": systemprompt}]
        # Beispiel: eigenes Modell - passe es an deine Bedürfnisse an
        # Oder nutze "gpt-4", "gpt-3.5-turbo", etc., falls verfügbar
        self.model = "gpt-4o-mini-2024-07-18"

    def chat(self):
        """
        Sendet den Prompt an das Chat-Interface und gibt den kompletten Antwort-String zurück.
        """
        # Nachricht (User-Prompt) zur Chat-Konversation hinzufügen
        self.context.append({"role": "user", "content": self.prompt})

        try:
            # API-Abfrage ausführen
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.context
            )
            response_content = response.choices[0].message.content

            # Antwort zur Konversation hinzufügen, falls wir kontextbasiert weiterarbeiten möchten
            self.context.append({"role": "assistant", "content": response_content})

            return response_content

        except Exception as e:
            print(f"Fehler bei der OpenAI-Anfrage: {e}")
            return ""

    def chat_with_streaming(self):
        """
        Optional: Interagiert mit OpenAI Chat Completion API und streamt die Antwort.
        """
        self.context.append({"role": "user", "content": self.prompt})
        print(f"User: {self.prompt}")
        print("AI: ", end="", flush=True)

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.context,
                stream=True
            )
            streamed_content = ""

            for chunk in response:
                delta = chunk.choices[0].delta
                content = getattr(delta, "content", "")
                if content:
                    print(content, end="", flush=True)
                    streamed_content += content

            print()  # Neue Zeile am Ende
            self.context.append({"role": "assistant", "content": streamed_content})

        except Exception as e:
            print(f"\nDEBUG: An error occurred during streaming: {e}")


def scrape_website(start_url, max_pages=50):
    """
    Crawlt die Website beginnend bei start_url.
    Limitierung auf max_pages, um Endlosschleifen zu vermeiden.
    Gibt ein Dictionary zurück: {URL: extrahierter_Text}.
    """
    visited = set()
    to_visit = [start_url]
    domain = urlparse(start_url).netloc
    scraped_data = {}

    while to_visit and len(visited) < max_pages:
        url = to_visit.pop(0)
        if url in visited:
            continue

        visited.add(url)
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200 and "text/html" in response.headers.get("Content-Type", ""):
                soup = BeautifulSoup(response.text, "html.parser")

                # Text extrahieren
                text = extract_text_from_soup(soup)
                scraped_data[url] = text

                # Interne Links sammeln
                for link in soup.find_all("a", href=True):
                    absolute_link = urljoin(url, link["href"])
                    # Prüfen, ob Link auf derselben Domain ist
                    if urlparse(absolute_link).netloc == domain:
                        if absolute_link not in visited and absolute_link not in to_visit:
                            to_visit.append(absolute_link)

        except requests.RequestException as e:
            print(f"Fehler beim Abrufen von {url}:\n{e}")

    return scraped_data


def extract_text_from_soup(soup):
    """
    Extrahiert reinen Text aus relevanten HTML-Tags.
    Entfernt Skripte, Styles und Kommentare.
    """
    # Entferne Skript- und Style-Tags
    for script_or_style in soup(["script", "style", "noscript"]):
        script_or_style.decompose()

    # Kommentare entfernen
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()

    # Text aus den relevanten Tags sammeln
    texts = []
    for tag in soup.find_all(["p", "h1", "h2", "h3", "li"]):
        if tag.get_text(strip=True):
            texts.append(tag.get_text(strip=True))

    # Alles in einen String
    full_text = "\n".join(texts)
    return full_text


def chunk_text(text, max_tokens=2000):
    """
    Teilt den Text in Blöcke auf, damit er nicht zu lang
    für die OpenAI-API wird.

    Hier sehr vereinfacht: 1 Token ~ 4 Zeichen als Daumenwert.
    """
    chunks = []
    approx_char_limit = max_tokens * 4
    start = 0
    while start < len(text):
        end = start + approx_char_limit
        chunk = text[start:end]
        chunks.append(chunk)
        start = end
    return chunks


def analyze_text_with_chatbot(text_chunk):
    """
    Baut auf der Chatbot-Klasse auf, um den Text SEO-technisch zu analysieren.
    Du kannst den Prompt beliebig anpassen.
    """
    system_prompt = "Du bist ein hochqualifizierter SEO-Experte. Du arbeitest für international erfolgreiche online marketing strategen. Deine Vorgesetzten bewundern dich für deine tiefgreifenden Kenntnisse auf dem Gebiet der SEO."
    user_prompt = (
        "1. Untersuche den folgenden Text auf Keyword-Optimierung, Lesbarkeit und mögliche SEO-Verbesserungen. "
        "2. Gib mir Handlungsempfehlungen, wie man diesen Inhalt optimieren kann. "
        "3. Schreibe den Text entsprechend der Handlungsempfehlungen um und gib ihn in der Ausgabe aus!"
        f"\n\n{text_chunk}"
    )

    # Initialisiere unsere Chatbot-Instanz mit System- und User-Prompt
    cb = Chatbot(systemprompt=system_prompt, prompt=user_prompt)

    # Wir lassen uns die Antwort zurückgeben
    analysis_result = cb.chat()
    return analysis_result


def main():
    # Beispiel-Start-URL:
    start_url = "https://www.rue-zahnspange.de"
    scraped_data = scrape_website(start_url, max_pages=10)

    seo_results = {}

    for url, text in scraped_data.items():
        if not text.strip():
            continue

        print(f"\n=== Analyzing {url} ===")
        text_chunks = chunk_text(text, max_tokens=2000)

        all_analyses = []
        for chunk_index, chunk in enumerate(text_chunks):
            print(f" - Sende Chunk {chunk_index+1}/{len(text_chunks)} an Chatbot ...")
            analysis = analyze_text_with_chatbot(chunk)
            all_analyses.append(analysis)

            # Warte kurz, um Rate Limits zu vermeiden
            time.sleep(1)

        # Gesamte Analyse zusammenfügen
        combined_analysis = "\n".join(all_analyses)
        seo_results[url] = combined_analysis

    # Ergebnisse ausgeben (oder in eine Datei speichern)
    print("\n=== SEO ANALYSE ERGEBNISSE ===")
    for url, analysis in seo_results.items():
        print(f"\nURL: {url}")
        print("Analyse:")
        print(analysis)


if __name__ == "__main__":
    main()


=== Analyzing https://www.rue-zahnspange.de ===
 - Sende Chunk 1/2 an Chatbot ...
 - Sende Chunk 2/2 an Chatbot ...

=== Analyzing https://www.rue-zahnspange.de/ ===
 - Sende Chunk 1/2 an Chatbot ...
 - Sende Chunk 2/2 an Chatbot ...

=== Analyzing https://www.rue-zahnspange.de/zahnspangen ===
 - Sende Chunk 1/1 an Chatbot ...

=== Analyzing https://www.rue-zahnspange.de/behandlungsablauf ===
 - Sende Chunk 1/1 an Chatbot ...

=== Analyzing https://www.rue-zahnspange.de/erwachsene ===
 - Sende Chunk 1/1 an Chatbot ...

=== Analyzing https://www.rue-zahnspange.de/uber-uns ===
 - Sende Chunk 1/1 an Chatbot ...

=== Analyzing https://www.rue-zahnspange.de/impressum ===
 - Sende Chunk 1/1 an Chatbot ...

=== Analyzing https://www.rue-zahnspange.de/datenschutz ===
 - Sende Chunk 1/5 an Chatbot ...
 - Sende Chunk 2/5 an Chatbot ...
 - Sende Chunk 3/5 an Chatbot ...
 - Sende Chunk 4/5 an Chatbot ...
 - Sende Chunk 5/5 an Chatbot ...

=== SEO ANALYSE ERGEBNISSE ===

URL: https://www.rue-zahns

In [10]:
import os
import requests
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin, urlparse
import time
import openai

# (Optional) Zur robusteren Erkennung von Encoding
# Könnte man auch verwenden, wenn man chardet installiert:
# import chardet


class Chatbot:
    """
    Diese Chatbot-Klasse nutzt die neue Methode client.chat.completions.create()
    aus openai>=1.0.0 über openai.OpenAI().
    """

    def __init__(self, systemprompt, prompt):
        # Neue Client-Initialisierung
        self.client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])
        self.systemprompt = systemprompt
        self.prompt = prompt
        self.context = [{"role": "system", "content": systemprompt}]
        # Beispiel: eigenes Modell - passe es an deine Bedürfnisse an.
        self.model = "gpt-4o-mini-2024-07-18"

    def chat(self):
        """
        Sendet den Prompt an das Chat-Interface und gibt den kompletten Antwort-String zurück.
        """
        # Nachricht (User-Prompt) zur Chat-Konversation hinzufügen
        self.context.append({"role": "user", "content": self.prompt})

        try:
            # API-Abfrage ausführen
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.context
            )
            response_content = response.choices[0].message.content

            # Antwort zur Konversation hinzufügen, falls wir kontextbasiert weiterarbeiten möchten
            self.context.append({"role": "assistant", "content": response_content})

            return response_content

        except Exception as e:
            print(f"Fehler bei der OpenAI-Anfrage: {e}")
            return ""

    def chat_with_streaming(self):
        """
        Optional: Interagiert mit OpenAI Chat Completion API und streamt die Antwort.
        """
        self.context.append({"role": "user", "content": self.prompt})
        print(f"User: {self.prompt}")
        print("AI: ", end="", flush=True)

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.context,
                stream=True
            )
            streamed_content = ""

            for chunk in response:
                delta = chunk.choices[0].delta
                content = getattr(delta, "content", "")
                if content:
                    print(content, end="", flush=True)
                    streamed_content += content

            print()  # Neue Zeile am Ende
            self.context.append({"role": "assistant", "content": streamed_content})

        except Exception as e:
            print(f"\nDEBUG: An error occurred during streaming: {e}")


def scrape_website(start_url, max_pages=50):
    """
    Crawlt die Website beginnend bei start_url.
    Limitierung auf max_pages, um Endlosschleifen zu vermeiden.
    Gibt ein Dictionary zurück: {URL: extrahierter_Text}.
    """
    visited = set()
    to_visit = [start_url]
    domain = urlparse(start_url).netloc
    scraped_data = {}

    while to_visit and len(visited) < max_pages:
        url = to_visit.pop(0)
        if url in visited:
            continue

        visited.add(url)
        try:
            response = requests.get(url, timeout=10)

            # Encoding-Probleme vermeiden:
            # Wir lassen requests das 'apparent_encoding' bestimmen und verwenden
            # es als .encoding-Fallback, falls nicht gesetzt.
            if not response.encoding:
                response.encoding = response.apparent_encoding or 'utf-8'

            if response.status_code == 200 and "text/html" in response.headers.get("Content-Type", ""):
                soup = BeautifulSoup(response.text, "html.parser")

                # Text extrahieren
                text = extract_text_from_soup(soup)
                scraped_data[url] = text

                # Interne Links sammeln
                for link in soup.find_all("a", href=True):
                    absolute_link = urljoin(url, link["href"])
                    # Prüfen, ob Link auf derselben Domain ist
                    if urlparse(absolute_link).netloc == domain:
                        if absolute_link not in visited and absolute_link not in to_visit:
                            to_visit.append(absolute_link)

        except requests.RequestException as e:
            print(f"Fehler beim Abrufen von {url}:\n{e}")

    return scraped_data


def extract_text_from_soup(soup):
    """
    Extrahiert reinen Text aus relevanten HTML-Tags.
    Entfernt Skripte, Styles und Kommentare.
    """
    # Entferne Skript- und Style-Tags
    for script_or_style in soup(["script", "style", "noscript"]):
        script_or_style.decompose()

    # Kommentare entfernen
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()

    # Text aus den relevanten Tags sammeln
    texts = []
    for tag in soup.find_all(["p", "h1", "h2", "h3", "li"]):
        if tag.get_text(strip=True):
            texts.append(tag.get_text(strip=True))

    # Alles in einen String
    full_text = "\n".join(texts)
    return full_text


def chunk_text(text, max_tokens=2000):
    """
    Teilt den Text in Blöcke auf, damit er nicht zu lang
    für die OpenAI-API wird.

    Hier sehr vereinfacht: 1 Token ~ 4 Zeichen als Daumenwert.
    """
    chunks = []
    approx_char_limit = max_tokens * 4
    start = 0
    while start < len(text):
        end = start + approx_char_limit
        chunk = text[start:end]
        chunks.append(chunk)
        start = end
    return chunks


def analyze_text_with_chatbot(text_chunk):
    """
    Baut auf der Chatbot-Klasse auf, um den Text SEO-technisch zu analysieren.
    Du kannst den Prompt beliebig anpassen.
    """
    system_prompt = "Du bist ein hochqualifizierter SEO-Experte."
    user_prompt = (
        "Untersuche den folgenden Text auf Keyword-Optimierung, Lesbarkeit und mögliche SEO-Verbesserungen. "
        "Gib mir Handlungsempfehlungen, wie man diesen Inhalt optimieren kann. "
        f"\n\n{text_chunk}"
    )

    # Initialisiere unsere Chatbot-Instanz mit System- und User-Prompt
    cb = Chatbot(systemprompt=system_prompt, prompt=user_prompt)

    # Wir lassen uns die Antwort zurückgeben
    analysis_result = cb.chat()
    return analysis_result


def main():
    # Beispiel-Start-URL:
    start_url = "https://www.rue-zahnspange.de"
    scraped_data = scrape_website(start_url, max_pages=10)

    seo_results = {}

    for url, text in scraped_data.items():
        if not text.strip():
            continue

        print(f"\n=== Analyzing {url} ===")
        text_chunks = chunk_text(text, max_tokens=2000)

        all_analyses = []
        for chunk_index, chunk in enumerate(text_chunks):
            # Debug-Ausgabe der Chunks
            print(f"\n--- CHUNK {chunk_index+1}/{len(text_chunks)} ---")
            print(chunk)  # Hier siehst du den gesamten Chunk (ACHTUNG: könnte viel sein)
            print("-----------------------------------------------")

            print(f" - Sende Chunk {chunk_index+1}/{len(text_chunks)} an Chatbot ...")
            analysis = analyze_text_with_chatbot(chunk)
            all_analyses.append(analysis)

            # Warte kurz, um Rate Limits zu vermeiden
            time.sleep(1)

        # Gesamte Analyse zusammenfügen
        combined_analysis = "\n".join(all_analyses)
        seo_results[url] = combined_analysis

    # Ergebnisse ausgeben (oder in eine Datei speichern)
    print("\n=== SEO ANALYSE ERGEBNISSE ===")
    for url, analysis in seo_results.items():
        print(f"\nURL: {url}")
        print("Analyse:")
        print(analysis)


if __name__ == "__main__":
    main()



=== Analyzing https://www.rue-zahnspange.de ===

--- CHUNK 1/2 ---
RÃ
Moderne Zahnspangen fÃ¼r ein gesÃ¼nderes LÃ¤cheln
Entdecken Sie die Praxis RÃÂ Zahnspange und unsere vielseitigen Behandlungen fÃ¼r Kinder und Jugendliche, die ein langanhaltendes und gesundes LÃ¤cheln ermÃ¶glichen.
Ãffnungszeiten
Hier finden Sie uns
Anbindungen
Vor unserer TÃ¼r befindet sich die Haltestelle MartinstraÃe, erreichbar Ã¼ber:
â
Bahn:Â 107, 108, U11
Bus: 142, 160, 161
â
Direkt gegenÃ¼ber der Praxis befindet sich ein gerÃ¤umiger Parkplatz.
Behandlungen fÃ¼r ein perfektes LÃ¤cheln
Unser Ziel ist es, Ihnen nicht nur ein Ã¤sthetisch ansprechendes LÃ¤cheln zu schenken, sondern auch Ihre gesamte Kiefergesundheit zu verbessern. Wir begleiten Sie auf jedem Schritt dieses Weges mit ProfessionalitÃ¤t und Sorgfalt.
Transparente AufklÃ¤rung
Wir setzen auf offene Kommunikation und ausfÃ¼hrliche Beratung, damit Sie jeden Schritt Ihrer Behandlung klar verstehen.
Spezialisiert auf Kinder und Jugendliche
Unser k

In [15]:
import os
import requests
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin, urlparse
import chardet


class WebsiteScraper:
    """
    Diese Klasse kümmert sich ausschließlich um das Sammeln und Extrahieren
    von Texten aus einer Website.
    """

    def __init__(self, start_url, max_pages=50):
        """
        :param start_url: Die Start-URL der Website, z.B. "https://www.example.com"
        :param max_pages: Maximale Anzahl Seiten, die gecrawlt werden.
        """
        self.start_url = start_url
        self.max_pages = max_pages

        # Hier speichern wir {URL: reiner_Text}
        self.scraped_data = {}

    def scrape_website(self):
        """
        Startet den Crawl-Vorgang, gefolgt von der Extraktion des Textes
        und dem Sammeln interner Links.
        """
        visited = set()
        to_visit = [self.start_url]
        domain = urlparse(self.start_url).netloc

        while to_visit and len(visited) < self.max_pages:
            url = to_visit.pop(0)
            if url in visited:
                continue
            visited.add(url)

            try:
                response = requests.get(url, timeout=10)

                # Rohdaten holen und Encoding per chardet bestimmen
                raw_data = response.content
                detected = chardet.detect(raw_data)
                encoding = "utf-8"
                text_data = raw_data.decode(encoding, errors="replace")

                # Nur weiterverarbeiten, wenn HTML-Content
                if (response.status_code == 200
                        and "text/html" in response.headers.get("Content-Type", "")):
                    soup = BeautifulSoup(text_data, "html.parser")

                    # Text extrahieren
                    text = self._extract_text_from_soup(soup)
                    self.scraped_data[url] = text

                    # Interne Links sammeln
                    for link in soup.find_all("a", href=True):
                        absolute_link = urljoin(url, link["href"])
                        if urlparse(absolute_link).netloc == domain:
                            if (absolute_link not in visited
                                    and absolute_link not in to_visit):
                                to_visit.append(absolute_link)

            except requests.RequestException as e:
                print(f"Fehler beim Abrufen von {url}:\n{e}")

    def _extract_text_from_soup(self, soup):
        """
        Extrahiert aus <p>, <h1>, <h2>, <h3>, <li> reinen Text,
        entfernt Script-/Style-/Noscript-Tags und Kommentare.
        """
        for script_or_style in soup(["script", "style", "noscript"]):
            script_or_style.decompose()

        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        texts = []
        for tag in soup.find_all(["p", "h1", "h2", "h3", "li"]):
            txt = tag.get_text(strip=True)
            if txt:
                texts.append(txt)

        return "\n".join(texts)

    def get_scraped_data(self):
        """
        Gibt das Dictionary {URL: Text} zurück.
        Du kannst damit arbeiten, Seiten filtern, etc.
        """
        return self.scraped_data


In [16]:
import openai
import time

class Chatbot:
    """
    Diese Chatbot-Klasse nutzt die neue Methode client.chat.completions.create()
    aus openai>=1.0.0 über openai.OpenAI().
    """

    def __init__(self, systemprompt, prompt):
        self.client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])
        self.systemprompt = systemprompt
        self.prompt = prompt
        self.context = [{"role": "system", "content": systemprompt}]
        self.model = "gpt-4o-mini-2024-07-18"  # Beispiel-Modell

    def chat(self):
        """
        Sendet den Prompt an das Chat-Interface und gibt den kompletten Antwort-String zurück.
        """
        self.context.append({"role": "user", "content": self.prompt})
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.context
            )
            response_content = response.choices[0].message.content
            self.context.append({"role": "assistant", "content": response_content})
            return response_content
        except Exception as e:
            print(f"Fehler bei der OpenAI-Anfrage: {e}")
            return ""


In [17]:
def chunk_text(text, max_tokens=2000):
    """
    Teilt den Text in Blöcke auf, damit er nicht zu lang
    für die OpenAI-API wird.
    Hier sehr vereinfacht: 1 Token ~ 4 Zeichen.
    """
    chunks = []
    approx_char_limit = max_tokens * 4
    start = 0
    while start < len(text):
        end = start + approx_char_limit
        chunk = text[start:end]
        chunks.append(chunk)
        start = end
    return chunks


In [18]:
def main():
    # 1. SCRAPING
    start_url = "https://www.rue-zahnspange.de"
    scraper = WebsiteScraper(start_url=start_url, max_pages=10)
    scraper.scrape_website()

    # Alle gescrapten Daten abrufen
    scraped_data = scraper.get_scraped_data()

    # 2. Sichten der Texte und Filtern
    #    Hier könntest du jetzt z. B. manuell prüfen, welche URLs wichtig sind.
    #    Wir geben einfach mal alle URLs aus:
    print("\n--- Gesammelte Seiten und Inhalte (gekürzt) ---")
    for url, text in scraped_data.items():
        print(f"\nURL: {url}")
        # Beispiel: Nur ersten 200 Zeichen zeigen
        print(f"Text (erst 200 Zeichen): {text[:200]}...")

    # Beispiel: Automatisches Filtern
    # Wir wollen z. B. keine Seiten analysieren, die 'impressum' im URL-Pfad enthalten:
    filtered_urls = [
        url for url in scraped_data
        if "impressum" not in url.lower()
    ]

    # 3. OPTIONAL: SEO-Analyse starten (für gefilterte Seiten)
    # Wir fragen den User oder machen es direkt (hier beispielhaft direkt).
    for url in filtered_urls:
        # Die gesamte Seite analysieren
        page_text = scraped_data[url]

        # 3.1 Chunken, um zu große Anfragen zu vermeiden
        text_chunks = chunk_text(page_text, max_tokens=2000)

        print(f"\n=== Analyzing {url} ===")
        all_analyses = []
        for i, chunk in enumerate(text_chunks):
            print(f" - Sende Chunk {i+1}/{len(text_chunks)} an Chatbot ...")

            # Prompt definieren (SEO)
            system_prompt = "Du bist ein hochqualifizierter SEO-Experte."
            user_prompt = (
                "Untersuche den folgenden Text auf Keyword-Optimierung, Lesbarkeit und mögliche SEO-Verbesserungen. "
                "Entwickle Handlungsempfehlungen, wie man diesen Inhalt optimieren kann."
                "Formuliere den Text entsprechende der Handlungsempfehlungen um und gebe mir als Ausgabe den Vergleich von Text chunk vorher zu Text chunk nachher. Hier ist der Text chunk: \n\n"
                f"{chunk}"
            )

            # ChatGPT aufrufen
            cb = Chatbot(systemprompt=system_prompt, prompt=user_prompt)
            analysis = cb.chat()
            all_analyses.append(analysis)

            # Warte kurz (Rate Limits, API-Kosten etc.)
            time.sleep(1)

        # 3.2 Fertige Analyse (alle Chunks zusammen)
        combined_analysis = "\n".join(all_analyses)
        print(f"\n--- SEO-Analyse für {url} ---")
        print(combined_analysis)


if __name__ == "__main__":
    main()



--- Gesammelte Seiten und Inhalte (gekürzt) ---

URL: https://www.rue-zahnspange.de
Text (erst 200 Zeichen): RÜ
Moderne Zahnspangen für ein gesünderes Lächeln
Entdecken Sie die Praxis RÜ Zahnspange und unsere vielseitigen Behandlungen für Kinder und Jugendliche, die ein langanhaltendes und gesundes Lächeln e...

URL: https://www.rue-zahnspange.de/
Text (erst 200 Zeichen): RÜ
Moderne Zahnspangen für ein gesünderes Lächeln
Entdecken Sie die Praxis RÜ Zahnspange und unsere vielseitigen Behandlungen für Kinder und Jugendliche, die ein langanhaltendes und gesundes Lächeln e...

URL: https://www.rue-zahnspange.de/zahnspangen
Text (erst 200 Zeichen): RÜ
Entdecken Sie unsere Zahnspangen
Jede Behandlung erfordert individuelle Zahnspangen verschiedenster Arten. Werfen Sie einen Blick darauf, was wir zu bieten haben.
Feste Zahnspangen
Verlässliche und...

URL: https://www.rue-zahnspange.de/behandlungsablauf
Text (erst 200 Zeichen): RÜ
Wir begleiten Sie bei jedem Schritt
Entdecken Sie unseren umf